In [25]:
import itertools
import numpy as np
import pandas as pd
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
# for neural net
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [12]:
data = pd.read_csv("data/combined_expression.csv")
data.head()
data.shape

(642, 16383)

In [13]:
selected_genes = pd.read_csv('cleaned/boruta-99-25-0.01.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [14]:
# retrieving proper columns
X = data.loc[:, selected_genes]
y = data['classification'].values

# scaling the data
scalar = MinMaxScaler()
x_scaled = scalar.fit_transform(X)

# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=0)

# Gridsearch for Input and Output Layer

In [15]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.25), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.125), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
model = KerasClassifier(build_fn=create_model)
epochs = [50, 75, 100, 150]
batches = [16, 32, 64, 128]
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adam', 'Adamax']
init = ['glorot_uniform', 'normal', 'uniform']
param_grid = dict(epochs=epochs, batch_size=batches,optimizer=optimizers,init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 240 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 27.8min finished


Train on 513 samples
Epoch 1/50
513/513 [==============================] - 2s 4ms/sample - loss: 0.6588 - accuracy: 0.6335
Epoch 2/50
513/513 [==============================] - 0s 179us/sample - loss: 0.6636 - accuracy: 0.6374
Epoch 3/50
513/513 [==============================] - 0s 178us/sample - loss: 0.6325 - accuracy: 0.6394
Epoch 4/50
513/513 [==============================] - 0s 176us/sample - loss: 0.6414 - accuracy: 0.6530
Epoch 5/50
513/513 [==============================] - 0s 176us/sample - loss: 0.6293 - accuracy: 0.6550
Epoch 6/50
513/513 [==============================] - 0s 176us/sample - loss: 0.6375 - accuracy: 0.6433
Epoch 7/50
513/513 [==============================] - 0s 178us/sample - loss: 0.6292 - accuracy: 0.6589
Epoch 8/50
513/513 [==============================] - 0s 173us/sample - loss: 0.6473 - accuracy: 0.6472
Epoch 9/50
513/513 [==============================] - 0s 173us/sample - loss: 0.6206 - accuracy: 0.6589
Epoch 10/50
513/513 [========================

In [11]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.748538 using {'batch_size': 64, 'epochs': 50, 'init': 'uniform', 'optimizer': 'Adagrad'}


In [12]:
grid_result.cv_results_

{'mean_fit_time': array([22.87440864, 29.32623951, 27.11062702, 24.87289794, 20.84396259,
        18.22900732, 26.27965927, 24.82301545, 21.94619497, 23.58998831,
        21.23158709, 28.46431748, 26.44677734, 23.70279694, 25.34283153,
        30.35232393, 40.48846563, 36.20284955, 34.93643999, 34.72135162,
        26.96035504, 38.37487062, 35.27403617, 31.93025732, 33.35870632,
        30.03306031, 38.8240184 , 35.94747686, 33.71194688, 34.08766039,
        38.66021053, 54.42402458, 51.30228074, 47.13256033, 43.75787473,
        32.6987474 , 47.30237158, 46.35322007, 49.22904126, 51.82448411,
        39.05789129, 49.82315334, 47.01785088, 45.01280864, 44.63239392,
        54.73623848, 77.51178463, 72.62631536, 69.90273364, 67.29407962,
        58.81166538, 74.67216094, 70.26548068, 62.4366796 , 63.05991546,
        56.51756843, 72.38511475, 69.1286943 , 59.17332363, 60.26967128,
        19.44160636, 24.24690302, 19.6103344 , 15.49522948, 15.72469044,
        12.50570965, 17.16207361, 

# Testing the Model

In [28]:
# model = KerasClassifier(build_fn=create_model, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'],optimizer=grid_result.best_params_['optimizer'],init=grid_result.best_params_['init'])
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=64, optimizer='Adagrad',init='uniform')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 4ms/sample - loss: 0.7010 - accuracy: 0.5234
Epoch 2/50
342/342 [==============================] - 0s 151us/sample - loss: 0.6811 - accuracy: 0.5965
Epoch 3/50
342/342 [==============================] - 0s 154us/sample - loss: 0.6670 - accuracy: 0.6111
Epoch 4/50
342/342 [==============================] - 0s 157us/sample - loss: 0.6624 - accuracy: 0.5994
Epoch 5/50
342/342 [==============================] - 0s 162us/sample - loss: 0.6671 - accuracy: 0.5819
Epoch 6/50
342/342 [==============================] - 0s 162us/sample - loss: 0.6613 - accuracy: 0.5994
Epoch 7/50
342/342 [==============================] - 0s 160us/sample - loss: 0.6501 - accuracy: 0.5789
Epoch 8/50
342/342 [==============================] - 0s 157us/sample - loss: 0.6444 - accuracy: 0.6170
Epoch 9/50
342/342 [==============================] - 0s 163us/sample - loss: 0.6488 - accuracy: 0.6345
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 4ms/sample - loss: 0.6762 - accuracy: 0.5906
Epoch 2/50
342/342 [==============================] - 0s 156us/sample - loss: 0.6481 - accuracy: 0.6520
Epoch 3/50
342/342 [==============================] - 0s 154us/sample - loss: 0.6270 - accuracy: 0.6520
Epoch 4/50
342/342 [==============================] - 0s 155us/sample - loss: 0.6230 - accuracy: 0.6462
Epoch 5/50
342/342 [==============================] - 0s 161us/sample - loss: 0.6303 - accuracy: 0.6608
Epoch 6/50
342/342 [==============================] - 0s 157us/sample - loss: 0.6287 - accuracy: 0.6404
Epoch 7/50
342/342 [==============================] - 0s 159us/sample - loss: 0.6096 - accuracy: 0.6579
Epoch 8/50
342/342 [==============================] - 0s 163us/sample - loss: 0.6088 - accuracy: 0.6550
Epoch 9/50
342/342 [==============================] - 0s 163us/sample - loss: 0.6054 - accuracy: 0.6404
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 4ms/sample - loss: 0.6857 - accuracy: 0.5351
Epoch 2/50
342/342 [==============================] - 0s 154us/sample - loss: 0.6393 - accuracy: 0.6667
Epoch 3/50
342/342 [==============================] - 0s 161us/sample - loss: 0.6270 - accuracy: 0.6754
Epoch 4/50
342/342 [==============================] - 0s 167us/sample - loss: 0.6132 - accuracy: 0.6930
Epoch 5/50
342/342 [==============================] - 0s 206us/sample - loss: 0.6230 - accuracy: 0.6871
Epoch 6/50
342/342 [==============================] - 0s 190us/sample - loss: 0.5968 - accuracy: 0.6930
Epoch 7/50
342/342 [==============================] - 0s 160us/sample - loss: 0.5973 - accuracy: 0.7105
Epoch 8/50
342/342 [==============================] - 0s 162us/sample - loss: 0.5933 - accuracy: 0.6988
Epoch 9/50
342/342 [==============================] - 0s 161us/sample - loss: 0.5946 - accuracy: 0.7193
Epoch 10/50
342/342 [========================

Baseline Accuracy: 70.37% (6.76%)


In [29]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/50
513/513 [==============================] - 2s 3ms/sample - loss: 0.6450 - accuracy: 0.6374
Epoch 2/50
513/513 [==============================] - 0s 167us/sample - loss: 0.6314 - accuracy: 0.6628
Epoch 3/50
513/513 [==============================] - 0s 166us/sample - loss: 0.6386 - accuracy: 0.6491
Epoch 4/50
513/513 [==============================] - 0s 171us/sample - loss: 0.6237 - accuracy: 0.6608
Epoch 5/50
513/513 [==============================] - 0s 168us/sample - loss: 0.6299 - accuracy: 0.6823
Epoch 6/50
513/513 [==============================] - 0s 173us/sample - loss: 0.6162 - accuracy: 0.6608
Epoch 7/50
513/513 [==============================] - 0s 168us/sample - loss: 0.6083 - accuracy: 0.6725
Epoch 8/50
513/513 [==============================] - 0s 174us/sample - loss: 0.6300 - accuracy: 0.6725
Epoch 9/50
513/513 [==============================] - 0s 169us/sample - loss: 0.6091 - accuracy: 0.6725
Epoch 10/50
513/513 [========================

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.65      0.91      0.76        75
           2       0.71      0.31      0.44        54

    accuracy                           0.66       129
   macro avg       0.68      0.61      0.60       129
weighted avg       0.67      0.66      0.62       129



In [31]:
print(confusion_matrix(y_test,test_predictions))

[[71  4]
 [39 15]]


In [32]:
model.model.save('models/hidden_4.h5')